In [ ]:
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.layers import Input, add, Conv1D, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, Conv1DTranspose, Reshape, ReLU, MaxPooling1D
import os
import shutil
from matplotlib import pyplot as plt

In [2]:
image_folders = os.listdir('sound images')
sound_folders = os.listdir('audio')

In [3]:
audio_names = []

In [ ]:
def load_audio(audio):
    final = []
    segment_length=16384
    for aud in audio:
        # if aud == 'gun':
        folder = 'C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/'+aud
        num = 0
        for file in os.listdir(os.path.join(folder)):
            file_name = os.path.join(folder, file)
            print(file_name)
            audio, _ = librosa.load(file_name, sr=16000)
            
            num += 1
           
            print(audio - np.min(audio))
            audio = (audio - np.min(audio)) / (np.max(audio) - np.min(audio)) * 2 - 1
    
            if len(audio) < 150912:
                padding = 150912 - len(audio)
                audio = np.pad(audio, (0, padding), 'constant')
            else:
                audio = audio[:150912]

            if len(audio) != 150912:
                print(f"Error: Audio length after processing is {len(audio)}, expected {150912}")
            # segments = [audio_padded[i:i+segment_length] for i in range(0, len(audio_padded), segment_length) if len(audio_padded[i:i+segment_length]) == segment_length]
    
            final.append(audio)
            audio_names.append(file_name)

            if num == 2:
                break
            
        
    return np.array(final)

def preprocess_audio(audio, segment_length=16384):
    audio = (audio - np.min(audio)) / (np.max(audio) - np.min(audio)) * 2 - 1
    # Segment audio
    # segments = [audio[i:i+segment_length] for i in range(0, len(audio), segment_length) if len(audio[i:i+segment_length]) == segment_length]
    return np.array(audio)

In [5]:
audio = os.listdir(os.path.join('C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/'))
audio_segments = load_audio(audio)

C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/baby\babycry-6473.1.157.mp3
[0.1695587  0.1695587  0.1695587  ... 0.16955869 0.16955851 0.16955936]
C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/baby\babycry-6473.1.mp3
[0.1695587  0.1695587  0.1695587  ... 0.16955869 0.16955851 0.16955936]
C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/cars\audi-v8-acceleration-sound-6067.156.mp3
[0.89905864 0.89905864 0.89905864 ... 0.89905864 0.89905864 0.89905864]
C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/cars\audi-v8-acceleration-sound-6067.157.mp3
[0.89905864 0.89905864 0.89905864 ... 0.89905864 0.89905864 0.89905864]
C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/fly\fly-129618.1.102.mp3
[0.7947285 0.7947285 0.7947285 ... 0.7947285 0.7947285 0.7947285]
C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/fly\fly-129618.1.mp3
[0.7947285 0.7947285 0.7947285 ... 0.7947285 0.7947285 0.7947285]
C:/Users/anang/OneDrive/Desktop/Image to Audio/audio/gun\9mm-pistol-shot-6

In [6]:
print(len(audio_names))

16


In [7]:
def load_and_preprocess_image(image_file):
    # Load the image from the file path
    final = []

    for img in image_file:
        # if img == 'gun':
        folder = 'C:/Users/anang/OneDrive/Desktop/Image to Audio/sound images/'+img
        num = 0
        for file_name in os.listdir(os.path.join(folder)):
            print(file_name)

            num += 1
            
            
            file = os.path.join(folder, file_name)
            # print(file)
            image = tf.io.read_file(file)
            # Decode the image (e.g., JPEG or PNG)
            image = tf.image.decode_jpeg(image, channels=3)
            # Resize the image to your desired dimensions
            image = tf.image.resize(image, [128, 128])
            # Normalize pixel values (optional)
            image = (image / 127.5) - 1
            
            final.append(image)

            if num == 2:
                break

            
    return np.array(final)

In [8]:
images = os.listdir(os.path.join('C:/Users/anang/OneDrive/Desktop/Image to Audio/sound images/'))
images = load_and_preprocess_image(images)

2457004-crying2.jpg
baby-1151351_1280.jpg
christian-bowen-856.jpeg
download (15).jpg
download (15).jpg
download (16).jpg
1Glock_17_b.jpg
2download (15).jpg
dark-1836972_1280.jpg
download (15).jpg
02ca3f80-34d4-414d-81d2-ef3064d1d369.jpg
07ebed3b64b7b965190d4025ffbd7f78.jpg
ai-generated-8111543_1280.png
amigos-2025167_1280.png
1animal-4004844_640.jpg
amur-tiger-4155922_640.jpg


In [9]:
print(images.shape)
print(audio_segments.shape)

(16, 128, 128, 3)
(16, 150912)


In [ ]:
data = tf.data.Dataset.from_tensor_slices((images, audio_segments))
data = data.batch(16).shuffle(2000)

In [ ]:
#MEAN SQUARED ERROR LOSS WAS USED IN PLACE O THIS

LAMBDA = 100

loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

In [ ]:
#DISCRIMINATOR IS UNUSED IN THIS TEMPLATE
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_loss = real_loss + generated_loss

  return total_loss

In [ ]:
#CODE FOR BUILDING THE GENERATOR

from tensorflow.keras import layers

def build_generator(latent_dim, output_shape):
    initializer = tf.random_normal_initializer(0., 0.02)
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(128, 128, 3)))
    model.add(layers.Conv2D(16, (2,2), strides=1, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(8, (2,2), strides=1, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(4, (2,2), strides=1, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())

    model.add(Dense(8*(output_shape[0] // 32))),
    model.add(Reshape((output_shape[0]//32, 8))),
    model.add(Conv1DTranspose(16, kernel_size=25, strides=4, padding='same', activation='relu')),
    model.add(Conv1DTranspose(1, kernel_size=25, strides=4, padding='same', activation='tanh')),
    
        
    return model

latent_dim = 100
output_shape = (301824, 1)

generator = build_generator(latent_dim, output_shape)
generator.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 8)         520       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 4)         132       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 4)        

In [ ]:
#COMPILING THE GENERATOR

generator.compile(optimizer='adam', loss='mse')

In [ ]:
#DISCRIMINATOR IS NOT USED IN THIS TEMPLATE

def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(128, 128, 3)))
    model.add(layers.Conv2D(16, kernel_size=15, strides=1, padding='same'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(8, kernel_size=15, strides=1, padding='same'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())

    initial_shape = (output_shape[0] // 128, 2) 
    model.add(layers.Dense(initial_shape[0] * initial_shape[1]))
    model.add(layers.ReLU())

    model.add(layers.Reshape((initial_shape[0] * initial_shape[1], 1)))

    model.add(layers.Conv1DTranspose(128, kernel_size=25, strides=2, padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Conv1DTranspose(64, kernel_size=25, strides=2, padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Conv1DTranspose(32, kernel_size=25, strides=2, padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Conv1DTranspose(16, kernel_size=25, strides=2, padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Conv1DTranspose(8, kernel_size=25, strides=2, padding='same'))
    model.add(layers.ReLU())

    model.add(layers.Conv1DTranspose(1, kernel_size=25, strides=2, padding='same', activation='tanh'))
    
  
    return model

discriminator = build_discriminator()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 16)      10816     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 64, 64, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 8)         28808     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 8)         0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 4716)             

In [ ]:
def train_wave_gan(generator, data, latent_dim, epochs=1000, batch_size=32):
    # half_batch = batch_size // 2
    for epoch in range(epochs):
        for input_image, audio in data.take(len(images)):

#PREVIOUS CODE INCLUDING THE USE OF THE DISCRIMINATOR
#         # if (step) % 1000 == 0:
            # num += 1
#         # discriminator.trainable=True

#         # idx = np.random.randint(0, data.shape[0], half_batch)
#         # real_samples = data[idx]
#         # print(real_samples.shape)
#         # real_labels = np.ones((half_batch, 1))

#         # noise = np.random.normal(0, 1, (half_batch, latent_dim))
#         # fake_samples = generator.predict(noise)
#         # fake_labels = np.zeros((half_batch, 1))

#         # d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
#         # d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)

#         # d_loss = np.add(d_loss_real, d_loss_fake) * 0.5

#         # discriminator.trainable = False

            # noise = np.random.normal(0, 1, (batch_size, latent_dim))
            # for i, o in zip(audio, input_image):
            #     num += 1
            #     sf.write(f'generated_audio{num}.wav', i, 16000)
            #     generate_images(o, num)
        
            
            # sf.write(f'generated_audio{num}.wav', generated_audio, 16000)
#         # valid_y = np.ones((batch_size, 1))
        
        #TRAINED WITHOUT THE DISCRIMINATOR
            gan_loss = generator.train_on_batch(input_image, audio)

        print(f"{epoch} [G loss: {gan_loss}]")

train_wave_gan(generator, data, latent_dim)


0 [G loss: 0.030673667788505554]
1 [G loss: 0.03056582808494568]
2 [G loss: 0.030465854331851006]
3 [G loss: 0.030372129753232002]
4 [G loss: 0.030268564820289612]
5 [G loss: 0.030142005532979965]
6 [G loss: 0.029984595254063606]
7 [G loss: 0.02979162335395813]
8 [G loss: 0.029561176896095276]
9 [G loss: 0.029295777902007103]
10 [G loss: 0.029005132615566254]
11 [G loss: 0.028704531490802765]
12 [G loss: 0.02841140143573284]
13 [G loss: 0.028130004182457924]
14 [G loss: 0.027835682034492493]
15 [G loss: 0.02748962864279747]
16 [G loss: 0.027076777070760727]
17 [G loss: 0.026614045724272728]
18 [G loss: 0.026121824979782104]
19 [G loss: 0.025595705956220627]
20 [G loss: 0.025004304945468903]
21 [G loss: 0.024312637746334076]
22 [G loss: 0.023505399003624916]
23 [G loss: 0.022600777447223663]
24 [G loss: 0.02162960171699524]
25 [G loss: 0.020606063306331635]
26 [G loss: 0.019558824598789215]
27 [G loss: 0.018558958545327187]
28 [G loss: 0.017649341374635696]
29 [G loss: 0.01680405810475

In [ ]:
def generate_images(img, num):
  # img = model(test_input, training=True)

  plt.figure(figsize=(5, 5))

  # display_list = [test_input[0], tar[0], img[0]]
  title = str(num)

  for i in range(1):
    plt.subplot(1, 1, i+1)
    plt.title(title)
    plt.imshow(img * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [27]:
# generator.fit(data, epochs=1000)

In [28]:
# gene = tf.keras.models.load_model('gn_da_audio.h5',
#                                   custom_objects={
#                                       "custom_loss": custom_loss,
#                                   }
#                                  )

In [18]:
def load_image(image_file):
    # Load the image from the file path

    file = image_file
    image = tf.io.read_file(file)
    # Decode the image (e.g., JPEG or PNG)
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to your desired dimensions
    image = tf.image.resize(image, [128, 128])
    # Normalize pixel values (optional)
    image = (image / 127.5) - 1
        

    return image

image = load_image("C://Users//anang//OneDrive//Desktop//Image to Audio//sound images//Tiger/1animal-4004844_640.jpg")

In [ ]:
def generate_audio(generator, latent_dim, length, image):
    # noise = np.random.normal(0, 1, (1, latent_dim))
    image = tf.expand_dims(image, axis=0)
    generated_audio = generator.predict(image)
    print(generated_audio)
    return generated_audio.flatten()

generated_audio = generate_audio(generator, latent_dim, output_shape[0], image)

# Save the generated audio using soundfile
import soundfile as sf
sf.write('generated_audio.wav', generated_audio, 16000)

1/1 [==============================] - 2s 2s/step
[[[ 0.01365701]
  [ 0.01492188]
  [ 0.01595006]
  ...
  [ 0.00161182]
  [-0.00713945]
  [ 0.00765297]]]


In [31]:
tf.keras.models.save_model(generator, 'new_gen_gen_audio.h5')

C:\Users\anang\AppData\Local\Temp\ipykernel_27108\2749786185.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(generator, 'new_gen_gen_audio.h5')


In [19]:
gene = tf.keras.models.load_model('new_gen_gen_audio.h5')